In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps, operator_average
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt, generate_Hamiltonian_with_occupation_number
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from pytenet.operation_thc import get_num_op_mpo


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [3]:
# H_correct = get_molecular_Hamiltonian_as_sparse_matrix(get_h1_spin(h1), get_g_spin(eri))
H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H_correct_NH3.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

In [4]:
H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
e ,v = sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [5]:
e_ground = e[0]
e_1st_ex = e[1]

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [6]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [7]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [8]:
#temp1 = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
#initial = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0])

initial = generate_single_state(8, [3, 3, 3, 3, 3, 0, 0, 0])

#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
# hartree_state = add_mps(temp1, temp2)
# hartree_state.orthonormalize('left')
# hartree_state.orthonormalize('right')

generate Krylov space:

In [9]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
#psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 150
trunc_tol = 1e-10
foldername_1 = f"/work_fast/ge49cag/code_datas/NH3_Krylov"
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

[1, 4, 9, 17, 23, 22, 11, 4, 1]
2
[1, 4, 14, 38, 57, 41, 15, 4, 1]
3
[1, 4, 16, 51, 97, 57, 16, 4, 1]
4


In [ ]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)

/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:208: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:107: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [ ]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

In [ ]:
#e_ritz_1, v_ritz_1 = solve_ritz_two_vec(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_ground, e_1st_ex, mpo_ref)

ground error (0.7940927865886493+0j)
first excited error (2.361285750591634+0j)
ground error (0.0742903956390677+0j)
first excited error (0.9561593790221963+0j)
ground error (0.006055777692210995+0j)
first excited error (0.6121926334759138+0j)
ground error (0.0023502472284206988+0j)
first excited error (0.42946095724316535+0j)
ground error (0.0021117996088833024+0j)
first excited error (0.14200213258493477+0j)
ground error (0.0020710751372803315+0j)
first excited error (0.06376993543304366+0j)


Restart:

In [ ]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 150
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/NH3_Krylov_restart_1"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)


In [ ]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:208: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:107: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [ ]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, mpo_ref)

In [ ]:
#e_ritz_2, v_ritz_2 = solve_ritz_two_vec(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, e_1st_ex, mpo_ref)

ground error (0.0737229314763681+0j)
first excited error (0.2149294491378413+0j)
ground error (0.009471039632131806+0j)
first excited error (0.06587347192504467+0j)
ground error (0.008334886514504092+0j)
first excited error (0.06111858653547131+0j)
ground error (0.008194723907308799+0j)
first excited error (0.05986899077619512+0j)
ground error (0.00819551552946507+0j)
first excited error (0.05987650229307917+0j)
ground error (0.008195515604015213+0j)
first excited error (0.05987650290393276+0j)


Restart:

In [ ]:
N_Krylov_3 = 30
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 150
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/NH3_Krylov_restart_2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)


In [ ]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)
H_reduced_3 = remain_only_tridiagonal_elements(H_reduced_3)

In [ ]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_ground, mpo_ref)

(0.0017008614715621206+0j)
(0.0014598163742221004+0j)


(0.0014448042806947115+0j)
(0.0014465827941876341+0j)


Restart:

In [ ]:
N_Krylov_4 = 20
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 150
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/NH3_Krylov_restart_2"
generate_krylov_space_in_disk(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4)


[1, 2, 4, 8, 16, 32, 64, 103, 123, 102, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 109, 129, 109, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 109, 129, 109, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 109, 129, 109, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
13
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]
14
[1, 2, 4, 8, 16, 32, 64, 109, 130, 109, 64, 32, 16, 8, 4, 2, 1]


In [ ]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)
H_reduced_4 = remain_only_tridiagonal_elements(H_reduced_4)

In [ ]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_ground, mpo_ref)

(0.001242530620800153+0j)
(0.0010893509091403075+0j)
(0.0010749644893479626+0j)


In [ ]:
np.linalg.eigh(H_reduced_4)

EighResult(eigenvalues=array([-67.46797558, -66.91092798, -65.99838787, -64.49687512,
       -62.99089116, -60.2149507 , -51.5303401 , -49.82418047,
       -48.25771974, -46.7813123 , -33.88833276, -32.52056411,
       -30.97456805, -29.34365196, -28.43720283]), eigenvectors=array([[-9.99957479e-01,  7.64715374e-03, -3.69788184e-03,
         2.38994410e-03,  1.15948899e-03, -7.74614162e-05,
        -8.90935742e-04, -1.67039253e-03,  1.18272035e-03,
        -7.44023235e-04, -2.98440977e-04,  3.23796770e-04,
         2.42499170e-04,  1.03328930e-04,  1.56017444e-04],
       [ 3.87280592e-03,  8.86387512e-02, -1.13101963e-01,
         1.47793206e-01,  1.08048925e-01, -1.16941874e-02,
        -2.95557368e-01, -6.13454779e-01,  4.72919913e-01,
        -3.20368366e-01, -2.08597312e-01,  2.35538444e-01,
         1.84203996e-01,  8.19971104e-02,  1.26752001e-01],
       [-3.92356566e-03, -2.27836674e-01,  2.77116845e-01,
        -3.32940978e-01, -2.22022129e-01,  1.97637328e-02,
         1.622